In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


To classify images using a recurrent neural network, we consider every image row as a sequence of pixels.Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 steps for every sample.

In [2]:
# Parameters 
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden =128 # hidden layer num of features
n_classes = 10 #  MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [3]:
def RNN(x, weights, biases):
    # Prepare data shape to match rnn function requirements
    # Current data input shape:( batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps,1)
    #print (x)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [4]:
# Launch the graph 
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # run optimization op ( backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y:batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x:batch_x, y:batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ', Minibatch_loss= '+\
                   "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")
    
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy: ", sess.run(accuracy, feed_dict={x:test_data, y:test_label}))

Iter 1280, Minibatch_loss= 1.800580, Training Accuracy= 0.39844


Iter 2560, Minibatch_loss= 1.518114, Training Accuracy= 0.41406


Iter 3840, Minibatch_loss= 1.078654, Training Accuracy= 0.65625


Iter 5120, Minibatch_loss= 1.050687, Training Accuracy= 0.67969


Iter 6400, Minibatch_loss= 0.837796, Training Accuracy= 0.70312


...

Iter 94720, Minibatch_loss= 0.213804, Training Accuracy= 0.95312


Iter 96000, Minibatch_loss= 0.088456, Training Accuracy= 0.97656


Iter 97280, Minibatch_loss= 0.089018, Training Accuracy= 0.98438


Iter 98560, Minibatch_loss= 0.124848, Training Accuracy= 0.96875


Iter 99840, Minibatch_loss= 0.103584, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy:  1.0
